# Atelier 2: Introduction à l'IA Agentique et au Model Context Protocol (MCP)

## Table des matières:

1. Comprendre l'IA Agentique
2. Introduction au Model Context Protocol (MCP)
3. Premier pas avec le MCP
4. Création d'un serveur MCP simple avec Flask
5. Création d'un client MCP
6. Exercice
7. Intégration avec un LLM

---

Bienvenue à cet atelier sur l'IA Agentique et les MCP! Aujourd'hui, nous allons explorer l'IA Agentique et créer nos propres servers MCP.

**Objectifs de l'atelier:**
1. Comprendre ce qu'est l'IA agentique
2. Explorer le Model Context Protocol (MCP)
3. Créer un serveur MCP simple
4. Développer un agent IA basique

In [ ]:
import os
import json
import requests
import time
from IPython.display import Markdown, display

## 1. Comprendre l'IA Agentique

### Qu'est-ce qu'un agent IA?

Un **agent IA** est un système qui peut:
- Percevoir son environnement
- Prendre des décisions autonomes
- Agir pour atteindre des objectifs spécifiques
- Utiliser des outils externes

Contrairement à un simple modèle de langage qui génère du texte, un agent IA peut interagir avec le monde extérieur et exécuter des actions concrètes.

### Composants d'un agent IA

<div style="text-align: center;">
    <img src="https://lh7-rt.googleusercontent.com/docsz/AD_4nXcM5IG8AX4rfhy8xP49NnRDK1X3KYfRpFtoU9XUKDXRRPDD3JhjMiJC12wk1r2D2aEhUZwvvW42WB4fn8cm5dgqlVIiSzB95fYO2GPATVK17Yk5DpU7Npzj1uLwoXLI4TVOFaDQWQ?key=UertKaKWfse9WlKbf6sBHwn1" alt="Architecture Agent IA" width="600"/>
    <p><em>Source: <a href="https://writesonic.com/blog/what-is-an-ai-agent">writesonic.com</a></em></p>
</div>

1. **Modèle de langage (LLM/SLM)**: Le "cerveau" qui comprend les requêtes et génère des réponses
2. **Système de planification**: Décompose les tâches complexes en étapes simples
3. **Interface avec des outils**: Permet d'interagir avec des API, bases de données, MCP, etc.
4. **Mémoire**: Stocke les conversations et le contexte

### Exemples d'applications des agents IA

- Assistant personnel qui peut envoyer des emails, planifier des réunions
- Agent de recherche qui collecte et synthétise des informations
- Assistant de programmation qui peut écrire et tester du code
- Agent d'automatisation qui exécute des workflows complexes

## 2. Introduction au Model Context Protocol (MCP)

### Qu'est-ce que le MCP?

Le **Model Context Protocol (MCP)** est un standard qui permet aux modèles de langage d'interagir avec des outils externes de manière structurée et sécurisée.

<div style="text-align: center;">
    <img src="https://miro.medium.com/v2/resize:fit:1100/format:webp/0*ydhvFVVoKyOfEHMi.png" alt="Architecture MCP" width="600"/>
    <p><em>Source: <a href="https://generativeai.pub/learn-mcp-servers-with-python-an-essential-guide-to-model-context-protocol-servers-97984f37eceb">generativeai.pub</a></em></p>
</div>



### Pourquoi utiliser le MCP?

- **Standardisation**: Interface commune pour tous les outils
- **Sécurité**: Contrôle précis des capacités du modèle
- **Extensibilité**: Facile d'ajouter de nouveaux outils
- **Transparence**: L'utilisateur peut voir quels outils sont utilisés

### Comment fonctionne le MCP?

1. Le modèle reçoit une description des outils disponibles
2. Lorsqu'il a besoin d'utiliser un outil, il génère une requête structurée
3. Le serveur MCP exécute l'outil et renvoie le résultat
4. Le modèle intègre ce résultat dans sa réponse

### Structure d'un outil MCP

In [ ]:
# Exemple de définition d'outil MCP
tool_example = {
    "name": "calculator",
    "description": "Effectue des calculs mathématiques",
    "parameters": {
        "type": "object",
        "properties": {
            "expression": {
                "type": "string",
                "description": "L'expression mathématique à évaluer"
            }
        },
        "required": ["expression"]
    }
}

print("Exemple de définition d'outil MCP:")
print(json.dumps(tool_example, indent=2))

## 3. Premiers pas avec MCP

Commençons par définir quelques outils simples que nous pourrons utiliser avec notre modèle de langage.

In [ ]:
# Définition de nos outils MCP
MCP_TOOLS = [
    {
        "name": "get_current_time",
        "description": "Obtient l'heure et la date actuelles",
        "parameters": {
            "type": "object",
            "properties": {},
            "required": []
        }
    },
    {
        "name": "calculate",
        "description": "Effectue un calcul mathématique",
        "parameters": {
            "type": "object",
            "properties": {
                "expression": {
                    "type": "string",
                    "description": "L'expression mathématique à évaluer"
                }
            },
            "required": ["expression"]
        }
    }
]

print("Nos outils MCP:")
print(json.dumps(MCP_TOOLS, indent=2))

In [ ]:
# Implémentation des fonctions d'outils
def get_current_time():
    """Renvoie l'heure et la date actuelles."""
    from datetime import datetime
    now = datetime.now()
    return now.strftime("%Y-%m-%d %H:%M:%S")

def calculate(expression):
    """Évalue une expression mathématique."""
    try:
        result = eval(expression)
        return f"Le résultat de {expression} est {result}"
    except Exception as e:
        return f"Erreur lors du calcul: {str(e)}"

# Fonction pour simuler l'appel d'un outil MCP
def simulate_mcp_call(tool_name, parameters):
    """Simule l'appel d'un outil MCP."""
    if tool_name == "get_current_time":
        return get_current_time()
    elif tool_name == "calculate":
        return calculate(parameters.get("expression", ""))
    else:
        return "Outil inconnu"

# Exemples d'utilisation
print("Démonstration d'appels MCP:")
print(f"get_current_time: {simulate_mcp_call('get_current_time', {})}")
print(f"calculate: {simulate_mcp_call('calculate', {'expression': '23 * 7 + 15'})}")

## 4. Création d'un serveur MCP simple avec Flask

Pour créer un véritable serveur MCP, nous allons utiliser Flask. Dans un environnement de production, vous exécuteriez ce code dans un fichier séparé, mais ici nous allons l'examiner dans le notebook.

### Code du serveur MCP

In [ ]:
from flask import Flask, request, jsonify
import json
from datetime import datetime

app = Flask(__name__)

# Définition des outils disponibles
TOOLS = [
    {
        "name": "get_current_time",
        "description": "Obtient l'heure et la date actuelles",
        "parameters": {
            "type": "object",
            "properties": {},
            "required": []
        }
    },
    {
        "name": "calculate",
        "description": "Effectue un calcul mathématique",
        "parameters": {
            "type": "object",
            "properties": {
                "expression": {
                    "type": "string",
                    "description": "L'expression mathématique à évaluer"
                }
            },
            "required": ["expression"]
        }
    },
    {
        "name": "get_weather",
        "description": "Obtient la météo pour une ville donnée",
        "parameters": {
            "type": "object",
            "properties": {
                "city": {
                    "type": "string",
                    "description": "Le nom de la ville"
                }
            },
            "required": ["city"]
        }
    }
]

@app.route('/tools', methods=['GET'])
def get_tools():
    return jsonify(TOOLS)

@app.route('/run_tool', methods=['POST'])
def run_tool():
    data = request.json
    tool_name = data.get('name')
    parameters = data.get('parameters', {})
    
    if tool_name == "get_current_time":
        result = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        return jsonify({"result": result})
    
    elif tool_name == "calculate":
        expression = parameters.get('expression')
        try:
            # Attention: eval peut être dangereux en production
            result = eval(expression)
            return jsonify({"result": f"Le résultat de {expression} est {result}"})
        except Exception as e:
            return jsonify({"error": f"Erreur de calcul: {str(e)}"}), 400
    
    elif tool_name == "get_weather":
        city = parameters.get('city')
        # Dans un cas réel, vous utiliseriez une API météo
        # Ici, nous simulons une réponse
        weather_data = {
            "Paris": "21°C, Ensoleillé",
            "New York": "18°C, Nuageux",
            "Tokyo": "25°C, Pluie légère",
            "London": "15°C, Brouillard"
        }
        result = weather_data.get(city, f"Données météo non disponibles pour {city}")
        return jsonify({"result": result})
    
    return jsonify({"error": "Outil non reconnu"}), 400

if __name__ == '__main__':
    app.run(debug=True, port=5000)

Pour exécuter ce serveur, vous devriez sauvegarder ce code dans un fichier `mcp_server.py` et l'exécuter avec `python mcp_server.py`.

## 5. Client MCP pour tester le serveur

Maintenant, créons un client simple pour interagir avec notre serveur MCP. Dans un environnement réel, ce client serait intégré à votre modèle de langage.

In [ ]:
import requests
import json

# URL du serveur MCP (à adapter selon votre configuration)
MCP_SERVER_URL = "http://localhost:5000"

# 1. Récupérer la liste des outils disponibles
def get_available_tools():
    response = requests.get(f"{MCP_SERVER_URL}/tools")
    if response.status_code == 200:
        return response.json()
    else:
        return f"Erreur: {response.status_code}"

# 2. Exécuter un outil
def run_tool(tool_name, parameters={}):
    data = {
        "name": tool_name,
        "parameters": parameters
    }
    response = requests.post(f"{MCP_SERVER_URL}/run_tool", json=data)
    if response.status_code == 200:
        return response.json()
    else:
        return f"Erreur: {response.status_code}"

# Exemples d'utilisation (à exécuter une fois le serveur lancé)
def test_mcp_client():
    print("Outils disponibles:")
    tools = get_available_tools()
    print(json.dumps(tools, indent=2))

    print("\\nHeure actuelle:")
    time_result = run_tool("get_current_time")
    print(time_result)

    print("\\nCalcul mathématique:")
    calc_result = run_tool("calculate", {"expression": "42 * 3 - 15"})
    print(calc_result)

    print("\\nMétéo à Paris:")
    weather_result = run_tool("get_weather", {"city": "Paris"})
    print(weather_result)

# Décommentez pour tester une fois le serveur lancé
# test_mcp_client()

**Note**: Pour exécuter ce client, vous devez d'abord lancer le serveur MCP dans un terminal séparé.

## 6. Exercice: Ajouter un nouvel outil MCP

Maintenant, c'est à votre tour! Créez un nouvel outil MCP qui permet de convertir des devises.

### Spécifications:
- Nom: `convert_currency`
- Description: "Convertit un montant d'une devise à une autre"
- Paramètres:
  - `amount`: Le montant à convertir (nombre)
  - `from_currency`: La devise source (ex: EUR, USD)
  - `to_currency`: La devise cible (ex: EUR, USD)

### Taux de conversion à utiliser (simplifiés):
- 1 EUR = 1.10 USD
- 1 EUR = 0.85 GBP
- 1 EUR = 160 JPY
- 1 USD = 0.91 EUR
- 1 USD = 0.77 GBP
- 1 USD = 145 JPY

Complétez le code ci-dessous:

In [ ]:
# Exercice: Ajouter un nouvel outil MCP pour la conversion de devises

# 1. Définir l'outil de conversion de devises
currency_tool = {
    "name": "convert_currency",
    "description": "Convertit un montant d'une devise à une autre",
    "parameters": {
        "type": "object",
        "properties": {
            "amount": {
                "type": "number",
                "description": "Le montant à convertir"
            },
            "from_currency": {
                "type": "string",
                "description": "La devise source (ex: EUR, USD, GBP, JPY)"
            },
            "to_currency": {
                "type": "string",
                "description": "La devise cible (ex: EUR, USD, GBP, JPY)"
            }
        },
        "required": ["amount", "from_currency", "to_currency"]
    }
}

# 2. Implémenter la fonction de conversion de devises
def convert_currency(amount, from_currency, to_currency):
    """
    Convertit un montant d'une devise à une autre.
    
    Paramètres:
    - amount: Le montant à convertir
    - from_currency: La devise source (EUR, USD, GBP, JPY)
    - to_currency: La devise cible (EUR, USD, GBP, JPY)
    
    Retourne:
    - Une chaîne de caractères indiquant le résultat de la conversion
    """
    # À COMPLÉTER: Implémentez la logique de conversion en utilisant les taux suivants:
    # 1 EUR = 1.10 USD
    # 1 EUR = 0.85 GBP
    # 1 EUR = 160 JPY
    # 1 USD = 0.91 EUR
    # 1 USD = 0.77 GBP
    # 1 USD = 145 JPY
    # 1 GBP = 1.18 EUR
    # 1 GBP = 1.30 USD
    # 1 GBP = 188 JPY
    # 1 JPY = 0.00625 EUR
    # 1 JPY = 0.0069 USD
    # 1 JPY = 0.0053 GBP
    
    # Votre code ici
    
    
    
    
    
    
    # Fin de votre code
    
    return f"{amount} {from_currency} = {converted_amount:.2f} {to_currency}"

# 3. Tester votre fonction
print("Tests de conversion de devises:")
# Décommentez ces lignes après avoir implémenté la fonction
# print(convert_currency(100, "EUR", "USD"))
# print(convert_currency(50, "USD", "JPY"))
# print(convert_currency(200, "GBP", "EUR"))

## 7. Intégration de l'outil de conversion de devises dans le serveur MCP

Maintenant que nous avons créé notre outil de conversion de devises, voyons comment l'intégrer dans notre serveur MCP.

Dans un serveur MCP réel, nous ajouterions cet outil à la liste des outils disponibles et implémenterions la fonction correspondante dans le gestionnaire de requêtes.

Voici comment cela se présenterait dans notre serveur Flask:

In [ ]:
# Code à ajouter au serveur MCP pour intégrer l'outil de conversion de devises

# 1. Ajouter l'outil à la liste TOOLS
"""
TOOLS.append({
    "name": "convert_currency",
    "description": "Convertit un montant d'une devise à une autre",
    "parameters": {
        "type": "object",
        "properties": {
            "amount": {
                "type": "number",
                "description": "Le montant à convertir"
            },
            "from_currency": {
                "type": "string",
                "description": "La devise source (ex: EUR, USD, GBP, JPY)"
            },
            "to_currency": {
                "type": "string",
                "description": "La devise cible (ex: EUR, USD, GBP, JPY)"
            }
        },
        "required": ["amount", "from_currency", "to_currency"]
    }
})
"""

# 2. Ajouter le gestionnaire dans la fonction run_tool
"""
@app.route('/run_tool', methods=['POST'])
def run_tool():
    data = request.json
    tool_name = data.get('name')
    parameters = data.get('parameters', {})
    
    # ... autres outils ...
    
    elif tool_name == "convert_currency":
        amount = parameters.get('amount')
        from_currency = parameters.get('from_currency')
        to_currency = parameters.get('to_currency')
        
        # Définir les taux de conversion
        rates = {
            "EUR": {"USD": 1.10, "GBP": 0.85, "JPY": 160, "EUR": 1.0},
            "USD": {"EUR": 0.91, "GBP": 0.77, "JPY": 145, "USD": 1.0},
            "GBP": {"EUR": 1.18, "USD": 1.30, "JPY": 188, "GBP": 1.0},
            "JPY": {"EUR": 0.00625, "USD": 0.0069, "GBP": 0.0053, "JPY": 1.0}
        }
        
        # Vérifier si les devises sont supportées
        if from_currency not in rates or to_currency not in rates:
            return jsonify({
                "error": f"Devise non supportée. Devises disponibles: {', '.join(rates.keys())}"
            }), 400
        
        # Effectuer la conversion
        conversion_rate = rates[from_currency][to_currency]
        converted_amount = amount * conversion_rate
        
        result = f"{amount} {from_currency} = {converted_amount:.2f} {to_currency}"
        return jsonify({"result": result})
    
    # ... autres outils ...
"""

## 8. Intégration avec un modèle de langage réel

Maintenant que nous avons créé notre serveur MCP et nos outils, intégrons-les avec un véritable modèle de langage. Nous allons utiliser:
- Soit le modèle local LiquidAI/llama.cpp que nous avons configuré dans le notebook 1
- Soit l'API OpenAI (avec une clé API partagée)

Cette intégration nous permettra de créer un véritable agent IA capable d'utiliser nos outils MCP pour répondre à des questions complexes.

In [ ]:
# Configuration pour l'intégration avec un modèle de langage

import os
import json
import requests
from dotenv import load_dotenv

# Configuration pour le modèle local (LiquidAI/llama.cpp)
from openai import OpenAI

# Client pour le modèle local
local_client = OpenAI(
    base_url="http://localhost:8080/v1",
    api_key="sk-no-key-required"  # Clé factice, llama.cpp n'en a pas besoin
)

# Fonction pour vérifier que le serveur local est en cours d'exécution
def check_local_server():
    try:
        base_url_str = str(local_client.base_url)
        
        # Supprimer "/v1" de l'URL de base pour accéder à "/models"
        if base_url_str.endswith("/v1/"):
            base_url_str = base_url_str[:-3]
            
        response = requests.get(f"{base_url_str}/models")
        if response.status_code == 200:
            print("✅ Connexion au serveur llama.cpp réussie!")
            return True
        else:
            print(f"❌ Le serveur a répondu avec le code {response.status_code}")
            return False
    except Exception as e:
        print(f"❌ Impossible de se connecter au serveur llama.cpp: {str(e)}")
        print("Assurez-vous que le serveur est en cours d'exécution sur http://localhost:8080")
        return False

# Vérifier si le modèle local est disponible
LOCAL_MODEL_AVAILABLE = check_local_server()


In [ ]:
# Configuration de la clé API OpenAI
import os
os.environ["OPENAI_API_KEY"] = "sk-votre-clé-api-ici"  # Remplacez par votre clé API
print("✅ Clé API OpenAI configurée")

In [ ]:
# Configuration pour OpenAI API
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
OPENAI_AVAILABLE = False

if OPENAI_API_KEY:
    try:
        # Configurer le client OpenAI
        openai_client = OpenAI(api_key=OPENAI_API_KEY)
        
        # Vérifier si la clé API est valide avec un appel minimal
        # La méthode list() ne prend pas d'argument limit dans les versions récentes
        response = openai_client.models.list()
        
        OPENAI_AVAILABLE = True
        print("✅ API OpenAI configurée et disponible")
    except Exception as e:
        print(f"❌ Erreur lors de la configuration de l'API OpenAI: {str(e)}")
else:
    print("❌ Clé API OpenAI non trouvée")

# Vérifier qu'au moins une option est disponible
if not LOCAL_MODEL_AVAILABLE and not OPENAI_AVAILABLE:
    print("⚠️ ATTENTION: Aucun modèle de langage n'est disponible. Veuillez configurer soit le modèle local, soit l'API OpenAI.")

In [ ]:
# Classe pour notre agent IA utilisant le serveur MCP
class MCPAgent:
    def __init__(self, use_openai=False):
        """
        Initialise l'agent IA avec le modèle de langage spécifié.
        
        Args:
            use_openai (bool): Si True, utilise l'API OpenAI. Sinon, utilise le modèle local.
        """
        self.use_openai = use_openai
        self.conversation_history = []
        self.mcp_server_url = "http://localhost:5000"
        
        # Vérifier si l'option choisie est disponible
        if use_openai and not OPENAI_AVAILABLE:
            raise ValueError("L'API OpenAI n'est pas disponible. Veuillez configurer la clé API ou utiliser le modèle local.")
        
        if not use_openai and not LOCAL_MODEL_AVAILABLE:
            raise ValueError("Le modèle local n'est pas disponible. Veuillez démarrer le serveur llama.cpp ou utiliser l'API OpenAI.")
        
        # Initialiser le client approprié
        if use_openai:
            self.client = openai_client
            print("Agent configuré pour utiliser l'API OpenAI")
        else:
            self.client = local_client
            print("Agent configuré pour utiliser le modèle local (LiquidAI/llama.cpp)")
        
        # Vérifier que le serveur MCP est disponible
        try:
            self.tools = self._get_tools()
            print(f"Serveur MCP détecté avec {len(self.tools)} outils disponibles")
        except Exception as e:
            print(f"Erreur lors de la connexion au serveur MCP: {str(e)}")
            print("Assurez-vous que le serveur MCP est en cours d'exécution sur " + self.mcp_server_url)
    
    def _get_tools(self):
        """Récupère la liste des outils disponibles depuis le serveur MCP."""
        response = requests.get(f"{self.mcp_server_url}/tools")
        if response.status_code == 200:
            return response.json()
        else:
            raise Exception(f"Erreur {response.status_code} lors de la récupération des outils")
    
    def _run_tool(self, tool_name, parameters):
        """Exécute un outil via le serveur MCP."""
        data = {
            "name": tool_name,
            "parameters": parameters
        }
        response = requests.post(f"{self.mcp_server_url}/run_tool", json=data)
        if response.status_code == 200:
            return response.json()
        else:
            raise Exception(f"Erreur {response.status_code} lors de l'exécution de l'outil {tool_name}")
    
    def process_query(self, user_query):
        """
        Traite une requête utilisateur et renvoie une réponse.
        
        Args:
            user_query (str): La requête de l'utilisateur
            
        Returns:
            str: La réponse de l'assistant
        """
        # Ajouter la requête à l'historique
        self.conversation_history.append({"role": "user", "content": user_query})
        
        # Récupérer les outils disponibles
        try:
            tools = self._get_tools()
        except Exception as e:
            return f"Erreur lors de la récupération des outils MCP: {str(e)}"
        
        # Convertir les outils MCP au format OpenAI
        openai_tools = []
        for tool in tools:
            openai_tools.append({
                "type": "function",
                "function": {
                    "name": tool["name"],
                    "description": tool["description"],
                    "parameters": tool["parameters"]
                }
            })
        
        # Préparer les messages pour l'API
        messages = self.conversation_history.copy()
        
        try:
            # Appeler l'API avec les outils
            response = self.client.chat.completions.create(
                model="gpt-3.5-turbo" if self.use_openai else "local-model",  # Le nom du modèle n'est pas important pour llama.cpp
                messages=messages,
                tools=openai_tools,
                tool_choice="auto"
            )
            
            # Récupérer le message de réponse
            message = response.choices[0].message
            
            # Vérifier si le modèle veut utiliser un outil
            if message.tool_calls:
                # Exécuter chaque appel d'outil
                for tool_call in message.tool_calls:
                    # Extraire les informations de l'outil
                    tool_name = tool_call.function.name
                    tool_params = json.loads(tool_call.function.arguments)
                    
                    print(f"Le modèle utilise l'outil: {tool_name}")
                    
                    # Exécuter l'outil via le serveur MCP
                    try:
                        tool_result = self._run_tool(tool_name, tool_params)
                    except Exception as e:
                        tool_result = {"error": f"Erreur lors de l'exécution de l'outil: {str(e)}"}
                    
                    # Ajouter l'appel d'outil à l'historique
                    self.conversation_history.append({
                        "role": "assistant",
                        "content": None,
                        "tool_calls": [{
                            "id": tool_call.id,
                            "type": "function",
                            "function": {
                                "name": tool_name,
                                "arguments": tool_call.function.arguments
                            }
                        }]
                    })
                    
                    # Ajouter le résultat de l'outil à l'historique
                    self.conversation_history.append({
                        "role": "tool",
                        "tool_call_id": tool_call.id,
                        "content": json.dumps(tool_result)
                    })
                
                # Obtenir la réponse finale du modèle
                final_response = self.client.chat.completions.create(
                    model="gpt-3.5-turbo" if self.use_openai else "local-model",
                    messages=self.conversation_history
                )
                
                final_content = final_response.choices[0].message.content
                self.conversation_history.append({"role": "assistant", "content": final_content})
                return final_content
            else:
                # Le modèle a répondu directement sans utiliser d'outil
                content = message.content
                self.conversation_history.append({"role": "assistant", "content": content})
                return content
                
        except Exception as e:
            error_msg = f"Erreur lors de l'appel au modèle: {str(e)}"
            print(error_msg)
            
            # Gestion de secours pour les modèles locaux qui ne supportent pas complètement l'API OpenAI
            if not self.use_openai:
                try:
                    print("Tentative de repli sur l'API de complétion simple...")
                    
                    # Construire un prompt qui explique les outils disponibles
                    tools_description = json.dumps(tools, ensure_ascii=False, indent=2)
                    
                    # Construire le prompt avec l'historique de conversation
                    conversation_prompt = ""
                    for message in self.conversation_history:
                        role = message["role"]
                        content = message.get("content")
                        if role == "user" and content:
                            conversation_prompt += f"Utilisateur: {content}\n\n"
                        elif role == "assistant" and content:
                            conversation_prompt += f"Assistant: {content}\n\n"
                    
                    # Construire le prompt final avec instructions pour utiliser les outils
                    system_prompt = f"""Tu es un assistant IA qui peut utiliser des outils externes.
                    Voici les outils disponibles:
                    {tools_description}
                    
                    Pour utiliser un outil, réponds au format suivant:
                    ```json
                    {{
                      "tool": "nom_de_l_outil",
                      "parameters": {{
                        "param1": "valeur1",
                        "param2": "valeur2"
                      }}
                    }}
                    ```
                    Si tu n'as pas besoin d'utiliser d'outil, réponds normalement. """

                    full_prompt = f"{system_prompt}\n\n{conversation_prompt}Assistant: "
                
                    # Appeler l'API de complétion
                    response = requests.post(
                        "http://localhost:8080/completion",
                        json={
                            "prompt": full_prompt,
                            "max_tokens": 1000,
                            "temperature": 0.7,
                            "stop": ["\nUtilisateur:", "\n\nUtilisateur:"]
                        }
                    )
                    
                    if response.status_code != 200:
                        return f"Erreur {response.status_code} lors de l'appel au modèle local: {response.text}"
                    
                    model_response = response.json().get("content", "")
                    
                    # Analyser la réponse pour détecter un appel d'outil
                    import re
                    tool_call_match = re.search(r'```json\s*({.*?})\s*```', model_response, re.DOTALL)
                    
                    if tool_call_match:
                        # Extraire et analyser l'appel d'outil
                        try:
                            tool_call = json.loads(tool_call_match.group(1))
                            tool_name = tool_call.get("tool")
                            tool_params = tool_call.get("parameters", {})
                            
                            print(f"Le modèle utilise l'outil: {tool_name}")
                            
                            # Exécuter l'outil via le serveur MCP
                            try:
                                tool_result = self._run_tool(tool_name, tool_params)
                                print(f"Résultat de l'outil: {json.dumps(tool_result)}")
                                
                                # Construire un nouveau prompt avec le résultat de l'outil
                                tool_prompt = f"{full_prompt}Je vais utiliser l'outil {tool_name}.\n\nRésultat de l'outil: {json.dumps(tool_result, ensure_ascii=False)}\n\nMaintenant, je vais répondre à la question: "
                                
                                # Appeler à nouveau le modèle avec le résultat de l'outil
                                final_response = requests.post(
                                    "http://localhost:8080/completion",
                                    json={
                                        "prompt": tool_prompt,
                                        "max_tokens": 1000,
                                        "temperature": 0.7,
                                        "stop": ["\nUtilisateur:", "\n\nUtilisateur:"]
                                    }
                                )
                                
                                if final_response.status_code == 200:
                                    final_content = final_response.json().get("content", "")
                                    self.conversation_history.append({"role": "assistant", "content": final_content})
                                    return final_content
                            except Exception as tool_error:
                                return f"Erreur lors de l'exécution de l'outil {tool_name}: {str(tool_error)}"
                        except json.JSONDecodeError:
                            pass
                    
                    # Si aucun appel d'outil n'est détecté ou s'il y a une erreur, retourner la réponse directe
                    self.conversation_history.append({"role": "assistant", "content": model_response})
                    return model_response
                    
                except Exception as fallback_error:
                    return f"{error_msg}\n\nErreur de repli: {str(fallback_error)}"
            
            return error_msg

In [ ]:
# Tester notre agent avec quelques requêtes

# Choisir le modèle à utiliser
USE_OPENAI = False  # Mettre à True pour utiliser OpenAI, False pour le modèle local

# Créer l'agent
try:
    agent = MCPAgent(use_openai=USE_OPENAI)
    
    # Fonction pour afficher les conversations de manière plus lisible
    from IPython.display import HTML, display
    
    def display_conversation(query, response):
        display(HTML(f"""
        <div style="background-color: #f0f0f0; padding: 10px; border-radius: 5px; margin-bottom: 10px;">
            <strong>Vous:</strong> {query}
        </div>
        <div style="background-color: #e8f4f8; padding: 10px; border-radius: 5px; margin-bottom: 20px;">
            <strong>Assistant:</strong> {response}
        </div>
        """))
    
    # Test 1: Question simple sur la date
    query1 = "Quelle est la date et l'heure actuelles?"
    print("Traitement de la requête: " + query1)
    response1 = agent.process_query(query1)
    display_conversation(query1, response1)
    
    # Test 2: Calcul mathématique
    query2 = "Peux-tu calculer 25 * 16 + 42?"
    print("Traitement de la requête: " + query2)
    response2 = agent.process_query(query2)
    display_conversation(query2, response2)

    # Test 3: Conversion de devises (notre nouvel outil)
    query3 = "Quel temps fait il sur Paris?"
    print("Traitement de la requête: " + query3)
    response3 = agent.process_query(query3)
    display_conversation(query3, response3)
    
    # Test 4: Conversion de devises (notre nouvel outil)
    query4 = "Combien valent 100 euros en dollars américains?"
    print("Traitement de la requête: " + query4)
    response4 = agent.process_query(query4)
    display_conversation(query4, response4)
    
except Exception as e:
    print(f"Erreur lors de l'initialisation ou de l'utilisation de l'agent: {str(e)}")
    print("\nVeuillez vérifier que:")
    print("1. Le serveur du modèle local est en cours d'exécution (si vous utilisez le modèle local)")
    print("2. La clé API OpenAI est correctement configurée (si vous utilisez OpenAI)")
    print("3. Les outils MCP sont correctement définis")

## 8. Conclusion

Dans ce notebook, nous avons:

1. **Abordé les concepts fondamentaux de l'IA agentique**
   - Définition et caractéristiques d'un agent IA
   - Architecture et composants d'un agent IA

2. **Exploré le Model Context Protocol (MCP)**
   - Structure et fonctionnement du MCP
   - Avantages de l'utilisation du MCP pour les agents IA

3. **Créé un serveur MCP simple**
   - Implémentation d'un serveur MCP avec Flask
   - Définition d'outils MCP pour différentes fonctionnalités

4. **Développé et intégré un nouvel outil MCP**
   - Création d'un outil de conversion de devises
   - Intégration de l'outil dans l'environnement MCP

5. **Intégré notre serveur MCP avec un modèle de langage réel**
   - Utilisation du modèle local LiquidAI/llama.cpp
   - Option d'utilisation de l'API OpenAI
   - Création d'un agent IA complet capable d'utiliser des outils

Cette approche nous permet de créer des agents IA puissants qui peuvent interagir avec le monde extérieur via des outils, tout en gardant le contrôle sur les capacités et les limites de ces agents.